# Recommended Joke Notebook

Today, we'll use some joke rating data to check out recommendating systems, in particular collaborative filtering:

Here's a link to the place where I got the data:
http://goldberg.berkeley.edu/jester-data/
This is an older, smaller dataset, bigger, newer ones are here: http://eigentaste.berkeley.edu/dataset/

Get the csv data here:
https://drive.google.com/file/d/0B-SYGEBtMGQlaDUyUXlIdl8tYmc/view?usp=sharing

Check out some jokes, and get recommended jokes based on your ratings:
http://eigentaste.berkeley.edu/

In [12]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import SparsePCA

from sklearn.cluster import AgglomerativeClustering 
from sklearn.cluster import KMeans 

from sklearn.decomposition import TruncatedSVD # another approach we are not taking today...


import pandas as pd
from dplython import (DplyFrame, X, diamonds, select, dfilter, sample_n,
    sample_frac, head, arrange, mutate, group_by, summarize, DelayFunction) 

In [11]:
# import joke data
# here's a link to it:
# https://drive.google.com/file/d/0B-SYGEBtMGQlaDUyUXlIdl8tYmc/view?usp=sharing

ff = "/Users/gregce/MIDS/DATASCI_W207/Class Sessions/Week 13/jester-data-1.csv" # you will need to edit this directory
f = open(ff)

#column_names = f.readline() # you'd needs this ordinarily

data = np.loadtxt(f, delimiter=",")

# NOTE ABOUT THE DATA STRUCTURE
# first column of data: # of jokes rated
# columns 2-101 of data: ratings for jokes no. 1 to 100; if rating == 99, no rating is available;

# we will re-code the no-rating to 0 for stability of the simple SVD approach we'll take later on

# first replace the actual zeroes with small random numbers
data[:, 1:][data[:, 1:] == 0] = (np.random.rand(sum(data==0).sum()) * 2.0 - 1.0) * 0.0000001
data[:, 1:][data[:, 1:] == 99] = 0

print data.shape

print data[0, :]

(24983, 101)
[ 74.    -7.82   8.79  -9.66  -8.16  -7.52  -8.5   -9.85   4.17  -8.98
  -4.76  -8.5   -6.75  -7.18   8.45  -7.18  -7.52  -7.43  -9.81  -9.85
  -9.85  -9.37   1.5   -4.37  -9.81  -8.5    1.12   7.82   2.86   9.13
  -7.43   2.14  -4.08  -9.08   7.82   5.05   4.95  -9.17  -8.4   -8.4   -8.4
  -8.11  -9.13  -9.03  -9.08  -7.14  -6.26   3.79  -0.1    3.93   4.13
  -8.69  -7.14   3.2    8.3   -4.56   0.92  -9.13  -9.42   2.82  -8.64
   8.59   3.59  -6.84  -9.03   2.82  -1.36  -9.08   8.3    5.68  -4.81   0.
   0.     0.     0.     0.     0.     0.    -9.42   0.     0.     0.    -7.72
   0.     0.     0.     0.     0.     0.     0.     0.     2.82   0.     0.
   0.     0.     0.    -5.63   0.     0.     0.  ]


# Step 0: Do some EDA on the data

How many jokes does an average person rate? What is most jokes rated? What is the fewest number of jokes rated?

How many ratings on average does each joke have?

What is a joke with the most ratings? Which is one with the fewest ratings?

What is the average rating for all jokes? (Be sure to exclude the 99 values!)

In [24]:
## fill this in with some code!!

##How many jokes does an average person rate? What is most jokes rated? What is the fewest number of jokes rated?
#np.mean(data[:,0])
# ~72 jokes

df = pd.DataFrame(data)

def col_ratings(df):
    for idx, column in enumerate(df):
        print(idx,np.mean(df[column]))
        
t = col_ratings(df)
t


(0, 72.46747788496178)
(1, 0.5956866669177919)
(2, 0.15172837531481534)
(3, 0.21088059881807175)
(4, -0.9053264219549539)
(5, 0.40345955251526106)
(6, 1.303932674209641)
(7, -0.42954208861859206)
(8, -0.6240351439010853)
(9, -0.3512128247474979)
(10, 1.0585682263727103)
(11, 1.577244125984062)
(12, 1.3007817316011678)
(13, -1.7627406636763494)
(14, 1.2583945082638304)
(15, -1.7057915382676698)
(16, -3.1038238001952947)
(17, -1.1150074050486023)
(18, -0.6350018012365267)
(19, 0.15419605333101288)
(20, -0.9338846415267542)
(21, 2.0904250890522023)
(22, 0.7452279549840695)
(23, 0.0875575391433539)
(24, -1.0725109074298047)
(25, 0.3185954449113595)
(26, 1.2227518712481515)
(27, 3.179734619525609)
(28, 1.4584609534152433)
(29, 2.966625305177604)
(30, -0.3140867790356137)
(31, 2.1701617099694475)
(32, 3.151566265050888)
(33, -0.9150978665612569)
(34, 0.7418352479624655)
(35, 3.0026201817498195)
(36, 3.3101136773132342)
(37, -0.9216451186807691)
(38, 1.1898627066703833)
(39, 0.986036104549778

# Unregularized SVD Based Collaborative Filtering

## step 1: full SVD

We'll do a simple though not very effective way at collaborative filtering: a simple SVD approach.

    SVD (singular value decomposition) is a way of decomposing a matrix into thing like eigenvectors and eigenvalues. It looks like this for a matrix M:

M = U * S * V

Here, if we suppose M is an n by p matrix:
U is an N by p matrix
S is a p by p diagonal matrix
V is a p by p matrix

In our problem, we have n users rating p jokes

In recommender problems, it has the interpretation as follows:
U is a representation of the users, as p features
V is a representation of the jokes, as p features (different from those in U)
S is a vector that gives the joint importance of both feature sets

In [12]:
# the full svd; note the first column is dropped since that has the number of jokes rated, we don't care about that
u, s, v = np.linalg.svd(data[:, 1:], full_matrices=False)

print u.shape
print s.shape
print v.shape

# these are (nearly) the same, that's the decomposition!
print data[:, 1:]
print np.dot(np.dot(u, np.diag(s)), v) # the full reconstruction

(24983, 100)
(100,)
(100, 100)
[[-7.82  8.79 -9.66 ...,  0.    0.    0.  ]
 [ 4.08 -0.29  6.36 ...,  0.34 -4.32  1.07]
 [ 0.    0.    0.   ...,  0.    0.    0.  ]
 ..., 
 [ 0.    0.    0.   ...,  0.    0.    0.  ]
 [ 0.    0.    0.   ...,  0.    0.    0.  ]
 [ 2.43  2.67 -3.98 ...,  0.    0.    0.  ]]
[[ -7.82000000e+00   8.79000000e+00  -9.66000000e+00 ...,   4.31654712e-13
    6.03739281e-13   8.28448421e-13]
 [  4.08000000e+00  -2.90000000e-01   6.36000000e+00 ...,   3.40000000e-01
   -4.32000000e+00   1.07000000e+00]
 [  1.73593778e-14  -1.16504029e-14   1.16989751e-14 ...,   1.09023901e-13
   -1.52100554e-13   6.92779167e-13]
 ..., 
 [ -1.36644168e-14  -1.17995891e-14   8.90780505e-15 ...,   1.13797860e-14
   -9.27036226e-15   1.22124533e-15]
 [ -2.16424101e-14  -5.87377369e-15   7.66400832e-15 ...,  -1.82076576e-14
    2.33146835e-15   5.55111512e-16]
 [  2.43000000e+00   2.67000000e+00  -3.98000000e+00 ...,  -5.99520433e-15
   -1.99840144e-15   5.66213743e-15]]


##  Step 2: Filling in values by truncating the SVD

The SVD above just recovers the original matrix. That is not very interesting, we want to get a new estimate for those missing values. To do this, we need to drop dimensions.

Inspect S, you should see that it is sorted in decreasing order. Write code to only take the first few dimensions in each of the projections. This means simply to replace the matrix S (you will have to construct it using np.diag) with a new matrix with many of the diagonal values zeroed out.

In [5]:
n_comp = 10

## write some code to take only the first n_comp values in s, then make an approximate reconstruction of the data

You should now have an approximation to the original matrix. Take the filled in values only and compare them to the non filled in values, do this for a few users and jokes. You might find it useful to write a function to inspect a row or column. Considering doing things like:

0. Count how many values are filled in, how many were already present for that row / column
1. Compare the range of the filled in values with those not filled in within the row or column
2. Compare the mean of the filled in values with those not filled in within the row or column

In [6]:
def inspect_filled_in_joke(ii):
    ## code this function to inspect the filled in bits of the approximate SVD
    print "joke ", ii, " is a great joke"
    
def inspect_filled_in_user(ii):
    ## code this function to inspect the filled in bits of the approximate SVD
    print "user ", ii, " is a funny user"

inspect_filled_in_joke(5)
inspect_filled_in_user(2)

joke  5  is a great joke
user  2  is a funny user


## Step 3: Let's actually use this thing. 

Write a function to recommend a joke (s)he's not rated to a user who has at least one unrated joke. Use it for a few people and print the results.

Use it on all users and make a histogram of the recommendations. Compare this to a histogram of missing values. Does this system recommend a joke very often? Does it make sense?

In [7]:
def recommend_to_user(ii, verbose = True):
    ## recommend a joke to a user
    if verbose:
        "Print out the recommendation"
    else:
        return -1

recs = []
for user_index in range(0, 25):
    recs.append(recommend_to_user(user_index))


## Step 4: Examining the joke and user spaces

The SVD gave us projections of both the users and the jokes. Let's see if those projections give us any insight.

Now, plot the first two rows (the projection dimensions appear on the rows here) of the joke projection matrix. Are there different types of jokes?

Plot the first two columns (the projection dimensions appear on the columns here) of the user projection matrix. Are there different types of users?

In [8]:
## just some scatterplots with plt.scatter(X, Y) should do it!